In [ ]:
import requests
import os
from concurrent.futures import ThreadPoolExecutor, wait, FIRST_COMPLETED
from tqdm import tqdm

# Map country code to country name (you can add more countries)
countries = {
"COM": "Comoros",
}

years = [2015, 2020]
ages = [60, 65, 70, 75, 80]
genders = ("f", "m")

root_folder = r"D:\Data\Population_Elderly"
max_workers = 3  # concurrent downloads

def ensure_folder_exists(path):
    if not os.path.exists(path):
        os.makedirs(path)

def get_remote_file_size(url):
    try:
        response = requests.head(url, timeout=15)
        response.raise_for_status()
        size = int(response.headers.get('Content-Length', 0))
        return size
    except requests.RequestException:
        return None

def download_file_with_progress(url, local_path, position):
    total_size = get_remote_file_size(url)
    if total_size is None:
        return False, f"Cannot get size for {url}, skipping."

    if os.path.exists(local_path):
        local_size = os.path.getsize(local_path)
        if local_size == total_size:
            return True, None  # Already downloaded and verified

    try:
        with requests.get(url, stream=True, timeout=60) as response:
            response.raise_for_status()
            with open(local_path, 'wb') as f, tqdm(
                total=total_size, unit='B', unit_scale=True,
                desc=os.path.basename(local_path), position=position,
                leave=True, miniters=1
            ) as pbar:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
                        pbar.update(len(chunk))
    except requests.RequestException as e:
        return False, f"Error downloading {url}: {e}"

    local_size = os.path.getsize(local_path)
    if local_size != total_size:
        return False, f"Size mismatch after download: {local_path}"

    return True, None

def main():
    all_files = []

    for country_code in countries.keys():
        for year in years:
            baseURL = f"https://data.worldpop.org/GIS/AgeSex_structures/Global_2000_2020/{year}/{country_code}/"
            country_folder = os.path.join(root_folder, country_code)
            ensure_folder_exists(country_folder)

            for gender in genders:
                for age in ages:
                    filename = f"{country_code.lower()}_{gender}_{age}_{year}.tif"
                    url = baseURL + filename
                    local_path = os.path.join(country_folder, filename)
                    all_files.append((url, local_path))

    total_downloaded = 0
    errors = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        position_map = {}

        # Start initial batch of downloads
        for i in range(min(max_workers, len(all_files))):
            url, path = all_files[i]
            future = executor.submit(download_file_with_progress, url, path, i)
            futures.append(future)
            position_map[future] = i

        next_file_index = max_workers

        while futures:
            done, futures = wait(futures, return_when=FIRST_COMPLETED)
            for future in done:
                pos = position_map[future]
                try:
                    success, msg = future.result()
                    if success:
                        total_downloaded += 1
                    else:
                        errors.append(msg)
                except Exception as exc:
                    errors.append(f"Exception in download task: {exc}")

                del position_map[future]

                if next_file_index < len(all_files):
                    url, path = all_files[next_file_index]
                    new_future = executor.submit(download_file_with_progress, url, path, pos)
                    futures.add(new_future)
                    position_map[new_future] = pos
                    next_file_index += 1

    print(f"\nDownload complete: {total_downloaded} files downloaded and verified out of {len(all_files)}.")
    if errors:
        print(f"\n{len(errors)} errors occurred during downloads:")
        for err in errors:
            print("-", err)

if __name__ == "__main__":
    main()



cmr_f_60_2015.tif:   0%|          | 0.00/215M [00:00<?, ?B/s]
cmr_f_60_2015.tif:   0%|          | 65.5k/215M [00:00<15:52, 226kB/s]
cmr_f_60_2015.tif:   0%|          | 156k/215M [00:00<13:11, 271kB/s] 
cmr_f_60_2015.tif:   0%|          | 172k/215M [00:00<20:41, 173kB/s]
cmr_f_60_2015.tif:   0%|          | 197k/215M [00:01<26:03, 137kB/s]
cmr_f_60_2015.tif:   0%|          | 221k/215M [00:01<30:24, 118kB/s]
cmr_f_60_2015.tif:   0%|          | 254k/215M [00:01<31:01, 115kB/s]
cmr_f_60_2015.tif:   0%|          | 287k/215M [00:02<33:14, 108kB/s]
cmr_f_60_2015.tif:   0%|          | 336k/215M [00:02<29:31, 121kB/s] 
cmr_f_60_2015.tif:   0%|          | 377k/215M [00:02<26:47, 134kB/s]
cmr_f_60_2015.tif:   0%|          | 410k/215M [00:02<24:10, 148kB/s]
cmr_f_60_2015.tif:   0%|          | 442k/215M [00:03<22:30, 159kB/s]
cmr_f_60_2015.tif:   0%|          | 516k/215M [00:03<17:23, 206kB/s]
cmr_f_60_2015.tif:   0%|          | 598k/215M [00:03<15:37, 229kB/s]
cmr_f_60_2015.tif:   0%|          | 6


Download complete: 40 files downloaded and verified out of 40.


In [ ]:
countries = {
    "AFG": "Afghanistan",
    "DZA": "Algeria",
    "AGO": "Angola",
    "ATG": "Antigua and Barbuda",
    "ARG": "Argentina",
    "BHS": "Bahamas",
    "BHR": "Bahrain",
    "BGD": "Bangladesh",
    "BRB": "Barbados",
    "BLZ": "Belize",
    "BEN": "Benin",
    "BTN": "Bhutan",
    "BOL": "Bolivia (Plurinational State of)",
    "BIH": "Bosnia and Herzegovina",
    "BWA": "Botswana",
    "BRA": "Brazil",
    "BRN": "Brunei Darussalam",
    "BFA": "Burkina Faso",
    "BDI": "Burundi",
    "KHM": "Cambodia",
    "CMR": "Cameroon",
    "CPV": "Cabo Verde",
    "CAF": "Central African Republic",
    "TCD": "Chad",
    "CHL": "Chile",
    "CHN": "China",
    "COL": "Colombia",
    "COM": "Comoros",
    "COG": "Congo",
    "CRI": "Costa Rica",
    "CUB": "Cuba",
    "DJI": "Djibouti",
    "DMA": "Dominica",
    "DOM": "Dominican Republic",
    "COD": "Democratic Republic of the Congo",
    "ECU": "Ecuador",
    "EGY": "Egypt",
    "SLV": "El Salvador",
    "GNQ": "Equatorial Guinea",
    "ERI": "Eritrea",
    "SWZ": "Eswatini",
    "ETH": "Ethiopia",
    "FJI": "Fiji",
    "GAB": "Gabon",
    "GMB": "Gambia",
    "GHA": "Ghana",
    "GRD": "Grenada",
    "GTM": "Guatemala",
    "GIN": "Guinea",
    "GNB": "Guinea-Bissau",
    "GUY": "Guyana",
    "HTI": "Haiti",
    "HND": "Honduras",
    "IND": "India",
    "IDN": "Indonesia",
    "IRN": "Iran (Islamic Republic Of)",
    "IRQ": "Iraq",
    "CIV": "Côte D'Ivoire",
    "JAM": "Jamaica",
    "JOR": "Jordan",
    "KEN": "Kenya",
    "KIR": "Kiribati",
    "KWT": "Kuwait",
    "LAO": "Lao People's Democratic Republic",
    "LBN": "Lebanon",
    "LSO": "Lesotho",
    "LBR": "Liberia",
    "LBY": "Libya",
    "MDG": "Madagascar",
    "MWI": "Malawi",
    "MYS": "Malaysia",
    "MDV": "Maldives",
    "MLI": "Mali",
    "MHL": "Marshall Islands",
    "MRT": "Mauritania",
    "MUS": "Mauritius",
    "FSM": "Micronesia (Federated States of)",
    "MNG": "Mongolia",
    "MAR": "Morocco",
    "MOZ": "Mozambique",
    "MMR": "Myanmar",
    "NAM": "Namibia",
    "NRU": "Nauru",
    "NPL": "Nepal",
    "NIC": "Nicaragua",
    "NER": "Niger",
    "NGA": "Nigeria",
    "PRK": "Democratic People's Republic of Korea",
    "OMN": "Oman",
    "PAK": "Pakistan",
    "PSE": "Palestine",
    "PAN": "Panama",
    "PNG": "Papua New Guinea",
    "PRY": "Paraguay",
    "PER": "Peru",
    "PHL": "Philippines",
    "QAT": "Qatar",
    "RWA": "Rwanda",
    "KNA": "Saint Kitts and Nevis",
    "LCA": "Saint Lucia",
    "VCT": "Saint Vincent and the Grenadines",
    "WSM": "Samoa",
    "STP": "Sao Tome and Principe",
    "SAU": "Saudi Arabia",
    "SEN": "Senegal",
    "SYC": "Seychelles",
    "SLE": "Sierra Leone",
    "SGP": "Singapore",
    "SLB": "Solomon Islands",
    "SOM": "Somalia",
    "ZAF": "South Africa",
    "SSD": "South Sudan",
    "LKA": "Sri Lanka",
    "SDN": "Sudan",
    "SUR": "Suriname",
    "SYR": "Syrian Arab Republic",
    "TJK": "Tajikistan",
    "TZA": "United Republic of Tanzania",
    "THA": "Thailand",
    "TLS": "Timor-Leste",
    "TGO": "Togo",
    "TON": "Tonga",
    "TTO": "Trinidad and Tobago",
    "TUN": "Tunisia",
    "TKM": "Turkmenistan",
    "UGA": "Uganda",
    "ARE": "United Arab Emirates",
    "URY": "Uruguay",
    "VUT": "Vanuatu",
    "VEN": "Venezuela (Bolivarian Republic Of)",
    "VNM": "Viet Nam",
    "YEM": "Yemen",
    "ZMB": "Zambia",
    "ZWE": "Zimbabwe",
}


In [ ]:
# Country list
countries = [
    "Brunei", "Burkina Faso",
    "Burundi", "Cambodia", "Cameroon", "Cabo Verde", "Central African Republic", "Chad",
    "Chile", "China", "Colombia", "Comoros", "Congo", "Costa Rica", "Cuba", "Djibouti",
    "Dominica", "Dominican Republic", "Democratic Republic of the Congo", "Ecuador", "Egypt",
    "El Salvador", "Equatorial Guinea", "Eritrea", "Eswatini", "Ethiopia", "Fiji", "Gabon",
    "Gambia", "Ghana", "Grenada", "Guatemala", "Guinea", "Guinea-Bissau", "Guyana", "Haiti",
    "Honduras", "India", "Indonesia", "Iran", "Iraq", "Ivory Coast", "Jamaica", "Jordan",
    "Kenya", "Kiribati", "Kuwait", "Laos", "Lebanon", "Lesotho", "Liberia", "Libya",
    "Madagascar", "Malawi", "Malaysia", "Maldives", "Mali", "Marshall Islands", "Mauritania",
    "Mauritius", "Micronesia", "Mongolia", "Morocco", "Mozambique", "Myanmar", "Namibia",
    "Nauru", "Nepal", "Nicaragua", "Niger", "Nigeria", "North Korea", "Oman", "Pakistan",
    "Palestine", "Panama", "Papua New Guinea", "Paraguay", "Peru", "Philippines", "Qatar",
    "Rwanda", "Saint Kitts and Nevis", "Saint Lucia", "Saint Vincent and the Grenadines",
    "Samoa", "Sao Tome and Principe", "Saudi Arabia", "Senegal", "Seychelles", "Sierra Leone",
    "Singapore", "Solomon Islands", "Somalia", "South Africa", "South Sudan", "Sri Lanka",
    "Sudan", "Suriname", "Syria", "Tajikistan", "Tanzania", "Thailand", "Timor-Leste", "Togo",
    "Tonga", "Trinidad and Tobago", "Tunisia", "Turkmenistan", "Uganda", "United Arab Emirates",
    "Uruguay", "Vanuatu", "Venezuela", "Vietnam", "Yemen", "Zambia", "Zimbabwe"
]

In [3]:
import os
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from pathlib import Path

# Map of ISO 3-letter country codes to full country names
countries = {
"COM": "Comoros"
}

# Settings
base_url_template = "https://data.worldpop.org/GIS/AgeSex_structures/Global_2000_2020/{year}/{code}/"
output_base_path = Path("D:/Data/Population_Elderly")
ages = [60, 65, 70, 75, 80]
genders = ["f", "m"]
years = [2015, 2020]
max_threads = 10

# Build download tasks
tasks = []
for code, name in countries.items():
    for year in years:
        for age in ages:
            for gender in genders:
                filename = f"{code.lower()}_{gender}_{age}_{year}.tif"
                url = base_url_template.format(year=year, code=code) + filename
                out_dir = output_base_path / name
                out_dir.mkdir(parents=True, exist_ok=True)
                out_path = out_dir / filename
                tasks.append((url, out_path))

# Individual file download
def download_file(url, out_path, pbar=None):
    try:
        with requests.get(url, stream=True, timeout=30) as r:
            r.raise_for_status()
            total = int(r.headers.get('content-length', 0))
            with open(out_path, 'wb') as f:
                for chunk in r.iter_content(8192):
                    if chunk:
                        f.write(chunk)
                        if pbar:
                            pbar.update(len(chunk))
        if pbar:
            pbar.set_description(f"✔ {out_path.name} ({out_path.stat().st_size // 1024} KB)")
    except Exception as e:
        if pbar:
            pbar.set_description(f"✖ {out_path.name} (Error: {str(e)[:30]})")

# Reverse-stacking visual bars
def download_all(tasks):
    print(f"\nStarting download of {len(tasks)} files using {max_threads} threads...\n")

    with ThreadPoolExecutor(max_workers=max_threads) as executor:
        bars = []
        futures = []

        for i, (url, out_path) in enumerate(tasks):
            pbar = tqdm(
                total=100,
                desc=f"{out_path.name}",
                position=max_threads - (i % max_threads) - 1,
                leave=True,
                dynamic_ncols=True,
                bar_format="{desc:<60} |{bar}| {percentage:3.0f}%",
            )
            bars.append(pbar)
            # Launch job with progress bar and proper position
            future = executor.submit(download_file, url, out_path, pbar)
            futures.append((future, pbar))

            # Limit to concurrent threads
            if len(futures) >= max_threads:
                for f, bar in futures:
                    f.result()
                    bar.close()
                futures.clear()

        # Final pending tasks
        for f, bar in futures:
            f.result()
            bar.close()

# Run the script
if __name__ == "__main__":
    download_all(tasks)



Starting download of 20 files using 10 threads...







































com_m_80_2015.tif                                            |          |   0%
















com_m_80_2015.tif                                            |          |   0%










































com_m_80_2015.tif                                            |          |   0%









































com_m_80_2015.tif                                            |          |   0%







































com_m_80_2015.tif                                            |          |   0%

















com_m_80_2015.tif                                            |          |   0%











































com_m_80_2015.tif                                            |          |   0%














































com_m_80_2015.tif                                            |          |   0%









































































com_m_80_2015.t